In [7]:
import math
import os
from tqdm.auto import tqdm

import torch
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader

from diffusers.optimization import get_scheduler
from google.cloud import storage

from y_sample_dataset3 import SamplingDataset

from model import FontDiffuserModel, FontDiffuserModelDPM, FontDiffuserDPMPipeline
from build import build_unet, build_style_encoder, build_content_encoder, build_ddpm_scheduler
from args import SampleArgs
from utils import x0_from_epsilon, reNormalize_img, normalize_mean_std, save_model, load_model

In [8]:
args = SampleArgs(itern=str(0))
unet = build_unet(args=args)
style_encoder = build_style_encoder(args=args)
content_encoder = build_content_encoder(args=args)
noise_scheduler = build_ddpm_scheduler(args)
storage_client = storage.Client(args.bucket_name)
bucket = storage_client.bucket(args.bucket_name)

Load the down block  DownBlock2D
Load the down block  MCADownBlock2D
The style_attention cross attention dim in Down Block 1 layer is 1024
The style_attention cross attention dim in Down Block 2 layer is 1024
Load the down block  MCADownBlock2D
The style_attention cross attention dim in Down Block 1 layer is 1024
The style_attention cross attention dim in Down Block 2 layer is 1024
Load the down block  DownBlock2D
Load the up block  UpBlock2D
Load the up block  StyleRSIUpBlock2D
Load the up block  StyleRSIUpBlock2D
Load the up block  UpBlock2D
Param count for Ds initialized parameters: 20591296
Get CG-GAN Style Encoder!
Param count for Ds initialized parameters: 1187008
Get CG-GAN Content Encoder!


In [9]:
savefd = f"/home/jupyter/ai_font/data/reports/exp0604/fontdiffuser_handcut"
os.makedirs(savefd, exist_ok=True)

In [10]:
target_font = "플레이브밤비"
dataset = SamplingDataset(args, target_font=target_font)
loader = DataLoader(dataset, batch_size=args.batchsize, shuffle=False, drop_last=False)

In [ ]:
itern = 30000
args = SampleArgs(itern=str(itern))
content_encoder.load_state_dict(load_model(bucket, args.content_encoder_path))
style_encoder.load_state_dict(load_model(bucket, args.style_encoder_path))
unet.load_state_dict(load_model(bucket, args.unet_path))

model = FontDiffuserModelDPM(
    unet=unet,
    style_encoder=style_encoder,
    content_encoder=content_encoder)

model = model.cuda()

train_scheduler = build_ddpm_scheduler(args=args)

pipe = FontDiffuserDPMPipeline(
        model=model,
        ddpm_train_scheduler=train_scheduler,
        model_type=args.model_type,
        guidance_type=args.guidance_type,
        guidance_scale=args.guidance_scale,
    )

# model.eval() 
results = []
pbar = tqdm(loader)
for data in pbar:
    content_image = data['content_img'].cuda()
    style_image = data['style_img'].cuda()
    contents = data['content']

    images = pipe.generate(
        content_images=content_image,
        style_images=style_image,
        batch_size=content_image.shape[0],
        order=args.order,
        num_inference_step=args.num_inference_steps,
        content_encoder_downsample_size=args.content_encoder_downsample_size,
        t_start=args.t_start,
        t_end=args.t_end,
        dm_size=args.content_image_size,
        algorithm_type=args.algorithm_type,
        skip_type=args.skip_type,
        method=args.method,
        correcting_x0_fn=args.correcting_x0_fn)
    for i in range(len(images)):
        path = f"{savefd}/{args.tag}__{target_font}__{contents[i]}.png"
        images[i].save(path)
        pbar.set_postfix(path=path)

  0%|          | 0/333 [00:00<?, ?it/s]